## Main Mega Project File
Pada file ini, kita akan melakukan beberapa hal terhadap data yang sudah kita scrapping. Data yang akan digunakan adalah data dari twit sebanyak 10000 dari twitter menggunakan api twint. 10000 baris data tersebut merupakan data dengan kandungan kata "islam" dan merupakan 10000 twit terakhir. Lebih tepatnya, data yang discrapping terdiri dari 10355 baris.

### Analysis Data

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

cwd = os.getcwd()
data_path = os.path.join(cwd, 'databases', 'store_twit_islam.json')

df = pd.read_json(data_path, lines = True)

df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1623980240056754176,1623980240056754176,2023-02-10 16:40:35 SE Asia Standard Time,2023-02-10,16:40:35,700,974330234689523715,thegoodofislam,For The Good of Islam,,...,,,,,,[],,,,
1,1623980213867782144,1623980213867782144,2023-02-10 16:40:29 SE Asia Standard Time,2023-02-10,16:40:29,700,129687395,voaislam,Voa Islam,,...,,,,,,[],,,,
2,1623980209002393601,1623979103069294592,2023-02-10 16:40:28 SE Asia Standard Time,2023-02-10,16:40:28,700,1306949180561092615,muslimu26805792,Nirob islam,,...,,,,,,"[{'screen_name': 'growing_bulls', 'name': 'Sol...",,,,
3,1623980198357045249,1623959852430110720,2023-02-10 16:40:26 SE Asia Standard Time,2023-02-10,16:40:26,700,1489154092844912644,pari_sa135,parisa,,...,,,,,,"[{'screen_name': 'devoted_islam', 'name': 'امـ...",,,,
4,1623980191448965120,1623965076557275136,2023-02-10 16:40:24 SE Asia Standard Time,2023-02-10,16:40:24,700,1087723151126159360,alimouhssen2,Ali Mouhssen 🇰🇲🇰🇲,,...,,,,,,"[{'screen_name': 'LaSueur_off', 'name': 'La Su...",,,,


In [5]:
df.shape

(10355, 36)

In [10]:
type(df)

pandas.core.frame.DataFrame

In [11]:
df[df.duplicated()]

TypeError: unhashable type: 'dict'